In [2]:
from transformers import MusicgenForConditionalGeneration
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")

/opt/conda/lib/python3.10/site-packages/transformers/models/encodec/modeling_encodec.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)
Config of the text_encoder: <class 'transformers.models.t5.modeling_t5.T5EncoderModel'> is overwritten by shared text_encoder config: T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 5

We can then place the model on our accelerator device (if available), or leave it on the CPU otherwise:

In [3]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device);

In [ ]:
print(device)

## Generation

MusicGen is compatible with two generation modes: greedy and sampling. In practice, sampling leads to significantly
better results than greedy, thus we encourage sampling mode to be used where possible. Sampling is enabled by default,
and can be explicitly specified by setting `do_sample=True` in the call to `MusicgenForConditionalGeneration.generate` (see below).

### Unconditional Generation

The inputs for unconditional (or 'null') generation can be obtained through the method `MusicgenForConditionalGeneration.get_unconditional_inputs`. We can then run auto-regressive generation using the `.generate` method, specifying `do_sample=True` to enable sampling mode:

In [ ]:
from datasets import Dataset
import pandas as pd

In [ ]:
df = pd.read_csv("data/musiccaps-train-data.csv")

In [ ]:
import os
df = df[df["ytid"].apply(lambda x: os.path.exists(f"data/wav_files/wav-48/{x}.wav"))]

In [8]:
df.head

<bound method NDFrame.head of              ytid  start_s  end_s  \
0     -0Gj8-vB1q4       30     40   
1     -0SdAVK79lg       30     40   
2     -0vPFx-wRRI       30     40   
3     -0xzrMun0Rs       30     40   
4     -1LrH01Ei1w       30     40   
...           ...      ...    ...   
5516  zw5dkiklbhE       15     25   
5517  zwfo7wnXdjs       30     40   
5518  zx_vcwOsDO4       50     60   
5519  zyXa2tdBTGc       30     40   
5520  zzNdwF40ID8       70     80   

                               audioset_positive_labels  \
0                          /m/0140xf,/m/02cjck,/m/04rlf   
1     /m/0155w,/m/01lyv,/m/0342h,/m/042v_gx,/m/04rlf...   
2                                   /m/025_jnm,/m/04rlf   
3                                    /m/01g90h,/m/04rlf   
4                                   /m/02p0sh1,/m/04rlf   
...                                                 ...   
5516                                /m/01sm1g,/m/0l14md   
5517                      /m/02p0sh1,/m/04rlf,/m/06j6

In [9]:
from tqdm import tqdm
import os
from pydub import AudioSegment
dataset_path = f"data/wav_files/wav-48"
for filename in tqdm(os.listdir(dataset_path)):
    if filename.endswith(('.mp3', '.wav', '.flac')):

        # move original file out of the way
        audio = AudioSegment.from_file(f"{dataset_path}/{filename}")

        # resample
        audio = audio.set_frame_rate(44100)
        audio.export(filename, format="wav")

100%|██████████████████████████████████████████████████████████████████████████████████████| 5079/5079 [01:20<00:00, 63.49it/s]


In [9]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [10]:
train.to_csv("train_labels.csv")
test.to_csv("test_labels.csv")

In [11]:
train_labels = pd.read_csv("train_labels.csv")
test_labels = pd.read_csv("test_labels.csv")

In [12]:
data = {
    "audio": train_labels["ytid"],
    "text": train_labels["caption"]
}

In [13]:
dataset = Dataset.from_dict(data)

In [14]:
dataset

Dataset({
    features: ['audio', 'text'],
    num_rows: 4063
})

In [ ]:
from transformers import AutoTokenizer, AutoFeatureExtractor
import torchaudio

tokenizer = AutoTokenizer.from_pretrained("facebook/musicgen-small")
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/musicgen-small")

def preprocess_function(examples):
    # Tokenize text
    tokens = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128, return_tensors="pt")
    # Load and preprocess audio
    print(examples["text"])
    print(examples["audio"])
    audio = [torchaudio.load(f"data/wav_files/wav-48/{path}.wav")[0] for path in [examples["audio"]]]
    return {
        "input_ids": tokens["input_ids"],
        "attention_mask": tokens["attention_mask"],
        "audio": audio  # Depending on the model's requirements, preprocess this further.
    }

processed_dataset = dataset.map(preprocess_function, batched=False)


Map:   0%|          | 0/4063 [00:00<?, ? examples/s]

This is a drum & bass electronic music piece. There is no singer. There is a strong and fuzzy feeling bass sound. The rhythmic background consists of a loud and high tempo electronic drum beat. The atmosphere is modern and energetic. This piece could be playing at a nightclub or a dance club.
FFQVVwFjy7s
The music excerpt features a male and a female voice singing the same melody in unison. Accompanying the singers is a sitar that plays the same note throughout - what is called a harmonic pedal. The atmosphere points towards a kind of meditation through repetitive singing, a kind of channeling state.
_h2rFVPCSPE
The low quality recording features a passionate female vocal, alongside harmonizing vocals, singing over wooden percussion, playing as the background music, while there are birds chirping, cows mooing and metallic impacts sounds. The recording is mono and noisy.
55qucu8Zs_I
This is a techno music piece played in the background of a car video. The sounds coming from the car such

This music is instrumental. The tempo is fast with a groovy didgeridoo harmony with percussion accompaniment. The music is upbeat, catchy, funky, unconventional, engaging, punchy and fun. This music is a Didgeridoo instrumental.
xCLzxuZE3yg
The music features a female voice singing with a reverb effect and out of tune. An acoustic guitar accompanies the singer with chords and arpeggios. In the background a synth sound that resembles an instrument with strings plays arpeggios.
v0tYHz5mk4I
This is a rap music piece played behind a rollerskating video. The sound of the skaters can be heard faintly throughout the recording. There is a male voice rapping at the forefront while other voices can be heard singing melodically in the background and ad-libbing occasionally. There is a mild keyboard playing the tune while a loud electronic drum beat is playing the rhythm. The atmosphere of this piece is groovy and urban.
_43OOP6UEw0
This song is recorded in poor quality. Two acoustic guitars are p

The low quality recording features a choir singing over punchy kick and snare hits, shimmering hi hats, groovy piano and groovy bass. In the first half, there are harmonizing female vocals singing, after which the male harmonizing vocals start singing. It sounds soulful, passionate and emotional. The recording is a bit noisy.
TokHdpvX7Es
This audio contains a catchy melody played by a brass instrument, maybe a saxophone and a keyboard. A horn is playing a bass in a lower register along with a drum. In the background you can hear people laughing. This is an amateur recording and of poor audio-quality.
JSqyTVjYY6k
This song contains an acoustic drum playing a simple rock groove with an e-bass playing a bassline along with the e-guitar strumming chords on the same notes as the bass. A deep male voice is singing, almost streaming sounding wild and loud. Then a synth pad comes in playing a melody panned to the left side of the speaker. This song may be playing at a rock festival.
AHmcuClSTL

The low quality recording features an acoustic guitar tuning tutorial. The recording is in mono and very noisy.
M27mIdPCZEY
The low quality recording features a groovy piano melody, mellow cello melody and brass melody. It sounds emotional and passionate, while the recording is noisy.
hBT0bbJl1dU
The Metal song features a manic solo electric guitar melody over shimmering hi hats, shimmering crash cymbals, double pedal kick hits, punchy snare, groovy distorted bass guitar and aggressive electric guitar riffs. It sounds aggressive, manic and energetic - like something you would jump to at the concerts.
OEuBITrf-kE
An ambient instrumental piece with no drums, big reverb and complex sound design elements. Tape hiss can be heard in the background and the vibe is hazy and warm with childlike wonder.
sgTZHSTnU40
This is a fast tempo song. The audio quality is inferior so the vocals are muddled making it difficult to comprehend if the vocalist is a male or female. The music is incoherent too b

...

This hip-hop song features a male voice rapping the main lyrics. This is accompanied by programmed percussion playing a simple beat. Hand claps are played on every alternate count. The synth plays a repetitive melody in loop. The bass plays the root notes of the chords in harmony with the synth. This is a moderate tempo song. This song can be played in a club.
vEMNk-lbGTE
The country music features a male voice singing while being backed by female vocals. The drums play a simple and repeating rhythm and together with the bass guitar drive the pulse of the music. An electric guitar plays short repeating musical phrases that incorporate the slide technique. In the background a group of strings fills the harmony and also provides a melody in the higher register.
VJ-dpTx_3Cg
A female singer sings this beautiful melody. The tempo is medium with a piano or keyboard accompaniment and no other instrumentation. The song is emotional and passionate like a love song. The audio quality is very poo

This piano song features a piano playing the backing chords. A violin plays the main melody. The melody is romantic but turns eerie at the end. There are no other instruments in this song. A male voice narrates two words toward the end of the song. This song can be played in a promotional video.
g0aOPWwNMFQ
This is a live performance classical music piece played by a chamber orchestra. It is in the style of a tarantella, which is a genre of Italian folk music. The strings section is playing the melody with a cello as the lead. The atmosphere is lively and elegant. It could be used in the soundtrack of a movie especially during a scene involving a pursuit, a face-off or a duel.
w0A-4EbkVz8
This audio contains an e-bass playing a melody along with a simple drum groove and other stringlike instruments. An e-guitar is playing a lead melody on top and in the background you can hear the crowd screaming. This is an amateur recording and of poor audio-quality. This song may be playing during a

This is an instrumental military march music piece. The uilleann pipes are playing the piece at a medium-to-high range. The atmosphere is epic. This piece could be used in the soundtrack of a historical drama during the scenes of a victory parade.
ZF8uHVu4Res
This is a mantra/meditation music piece. There is a male and a female vocal chanting in a devotional manner. There is a tambura melody going on in the background while certain percussive elements at higher and lower ranges can be heard in the rhythmic structure. The atmosphere is hypnotic. This piece could be used in Hindu religious events.
VuWr1HXHoZg
This is a remix of an Indian folk music piece. There is a male vocalist singing melodically and passionately. The rhythmic background is full of percussive elements that were emphasized by the use of electronic drums. The atmosphere is lively and jovial. This piece could be played in dance parties and also in contemporary dancing courses as an accompaniment piece.
_mQ6KuA2p6k
This i

The low quality recording features a classical song that consists of an aggressive brass melody, punchy percussion hits, mellow low percussions and hollow flute melody playing over it. It sounds vintage, suspenseful, reverberant, intense and the recording is noisy.
vBw99ghST1g
This is an emotional ballad played on the theremin. The audio quality is poor, but still we feel the sad and sombre effects of the melody played by the performer.
sL01xTmV_Fc
Here we get to hear a hardcore metal piece. This is characterised, in part, by the rapid action on the kick drum - with notes being played at a rate of sixteenth notes on the kick. The drummer also frequents the ride cymbal. The guitar is grungy, gritty, fuzzy, heavy, deep, hardcore.
5KvjUzQbMT4
This is the recording of a jazz improvisation performance. It is in an instrumental piece performed with a keyboard that utilizes a classical piano sound. The chords are being played gently which creates a calming atmosphere. Due to various chords an

The Country Blues song features a flat male vocal singing over acoustic guitar melody, accordion and a sustained brass section melody. It sounds very noisy, vintage and therefore nostalgic and sentimental - like something you would hear in an old movie.
V5xL8hLFY58
This pop song features a female voice singing the main melody. This is accompanied by male and female backing voices. The voices sing a repetitive melody. This is accompanied by percussion playing a simple beat. Trumpets play a backing melody. The bass plays a groovy descending bassline. The mood of this song is happy. This song can be played at a club.
EtmKuWPpjG0
The low quality recording features a loud didgeridoo modulating low tone. It is in mono and it sounds buzzy and kind of weird.
WoXgPQQjcJU
The low quality recording features a pop folk song that consists of passionate female vocal singing over repetitive flute licks, stuttering hi hats, groovy piano chords, claps and a riser sweep. It sounds energetic and kind of 

In [ ]:
processed_dataset

In [47]:
processed_dataset["input_ids"]

KeyError: "Column input_ids not in the dataset. Current columns in the dataset: ['audio', 'text']"

In [41]:
!ls data/wav-48

-0Gj8-vB1q4.wav  EptdhC17avY.wav  UD0pisqyLKA.wav  id5ibIqjRto.wav
-0SdAVK79lg.wav  EremEyLrdUY.wav  UDN11Q90Fa4.wav  idUZsNLnyDg.wav
-0vPFx-wRRI.wav  Es9FNjZ-SHI.wav  UDS6PrY9ZIM.wav  ieEPKa3HiGo.wav
-0xzrMun0Rs.wav  EsHXnkZ_W2c.wav  UDTsCTLkZzc.wav  iePMcLYozYY.wav
-1LrH01Ei1w.wav  EsssGCL-Axw.wav  UDg9vuBDtVU.wav  ihCl2ImrOYE.wav
-1OlgJWehn8.wav  EtmKuWPpjG0.wav  UEOUXjX5R2I.wav  ihJT65fZaHY.wav
-1UWSisR2zo.wav  Euu6zlJQSD0.wav  UEel3wTf0Sk.wav  ik-fXNjxw58.wav
-3Kv4fdm7Uk.wav  EwDiNj_5PEg.wav  UFyOGqmITjM.wav  ikEuQPSBY-0.wav
-4NLarMj4xU.wav  EwoCbcSXlSM.wav  UGEqY_NTMpI.wav  ikJKSqnTylI.wav
-4SYC2YgzL8.wav  Ex0ukzO5z9M.wav  UHvYrO1IGCc.wav  imIFtW4O5S0.wav
-5FoeegAgvU.wav  Ex18Xwznj60.wav  UIOnnpaqBy8.wav  iqEQBCrOLWc.wav
-5f6hjZf9Yw.wav  ExghbCGRBx0.wav  UJA5AWbt6HM.wav  iqOPJWWKo90.wav
-5xOcMJpTUk.wav  EyO5vB4eqo0.wav  UJAk1nNdo1I.wav  itT0_RhSipQ.wav
-6HBGg1cAI0.wav  EzP7PB2x670.wav  ULHPhjaJ6p0.wav  ivymRS3iEZk.wav
-6QGvxvaTkI.wav  Ezodz2aZnzQ.wav  ULXbXpLcoVA.wav  izVCz3P43p8

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
model.config.audio_encoder.sampling_rate

48000

In [6]:
audio_values.size()

torch.Size([1, 1, 489600])

Or save them as a `.wav` file using a third-party library, e.g. `scipy` (note here that we also need to remove the channel dimension from our audio tensor):

In [7]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./musicgen-musiccaps-finetuned-checkpoints",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    warmup_steps=500,
    learning_rate=5e-5,
    logging_dir='./logs',
    logging_steps=100,
)